## Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.




### Generate maps to visualize your neighborhoods and how they cluster together.

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
t_data = pd.read_csv('toronto_data_geo.csv')
t_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
1,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
2,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
3,M9N,York,Weston,43.706876,-79.518188
4,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846


In [4]:
address = 'Toronto'
geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of {} and {}, {}.'.format(address, latitude, longitude))

The geographical coordinate of Toronto and 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top

In [5]:
toronto_map = folium.Map(location = [latitude, longitude], zoom_start=10)

for lat, long, borough, neigh in zip(t_data['Latitude'], t_data['Longitude'], t_data['Borough'], t_data['Neighborhood']):
    label = '{}, {}'.format(neigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        colors='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)

toronto_map

### To check how many boroughs and neighborhoods in the dataset

In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(t_data['Borough'].unique()),
        t_data.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### To check which boroughs appeared most in the dataset

In [7]:
t_data['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

### To simplify the map above by exploring the `borough` with highest count in the dataset and segment and cluster that  is the neighborhood of `North York`


### Now is time to slice the original dataframe and create a new dataframe of the `North York` data

In [8]:
northyork_data = t_data[t_data['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.drop(['Postal Code'], axis=1)
northyork_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
3,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923
4,M2R,North York,Willowdale West,43.782736,-79.442259


#### Let get the geographical coordinates of `North York`

In [9]:
address = 'North York'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.


#### Lets visualize `North York` and the neighborhoods in it

In [10]:
# create map of North York using latitude and longitude values
northyork_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, long, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(northyork_map)  
    
northyork_map

#### Foursquare credentials

In [11]:
CLIENT_ID = 'TF5GWD1JC4MTFSRIJBNHTX225TCKIBO4IBZDAWY20U1KKEV5' # your Foursquare ID
CLIENT_SECRET = '3F0PHOPC4O0R140AMIPJP4FCMEW30EGWSAIQGQO5HEKXHTA0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TF5GWD1JC4MTFSRIJBNHTX225TCKIBO4IBZDAWY20U1KKEV5
CLIENT_SECRET:3F0PHOPC4O0R140AMIPJP4FCMEW30EGWSAIQGQO5HEKXHTA0


### Let's explore the second neighborhood in the `North York` dataframe

Get the name of the neighborhood and the latitude and longitude

In [12]:
northyork_data.loc[1, 'Neighborhood']

'Victoria Village'

In [13]:
neighborhood_latitude = northyork_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northyork_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = northyork_data.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Victoria Village are 43.7258823, -79.31557159999998.


### Using the Foursquare credential to get the top 100 venues that are in `Willowdale South` within the radius of 500 meters.

Create the GET request URL

In [14]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=TF5GWD1JC4MTFSRIJBNHTX225TCKIBO4IBZDAWY20U1KKEV5&client_secret=3F0PHOPC4O0R140AMIPJP4FCMEW30EGWSAIQGQO5HEKXHTA0&v=20180605&ll=43.7258823,-79.31557159999998&radius=500&limit=100'

Send the GET request and examine the results

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca1403d351e3d4c7b6a8549'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bermondsey',
  'headerFullLocation': 'Bermondsey, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.730382304500004,
    'lng': -79.30935618239715},
   'sw': {'lat': 43.7213822955, 'lng': -79.32178701760282}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c633acb86b6be9a61268e34',
       'name': 'Victoria Village Arena',
       'location': {'lat': 43.72348055545508,
        'lng': -79.31563520925143,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72348055545508,
          'lng': -7

Check length of the json data

In [16]:
len(results)

2

The information is in the items key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a `pandas` dataframe

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Tim Hortons,Coffee Shop,43.725517,-79.313103
2,Portugril,Portuguese Restaurant,43.725819,-79.312785
3,Eglinton Ave E & Sloane Ave/Bermondsey Rd,Intersection,43.726086,-79.313620
4,Pizza Nova,Pizza Place,43.725824,-79.312860


Lets know how many venues were returned by Foursquare?

In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


# Explore Neighborhood in `North York`

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Write the code to run the bove function on each neighborhood and create a new dataframe called `northyork_venues`.

In [21]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )

Fairview, Henry Farm, Oriole
Victoria Village
Silver Hills, York Mills
Flemingdon Park, Don Mills South
Willowdale West
Emery, Humberlea
Bayview Village
Downsview Central
Humber Summit
Bedford Park, Lawrence Manor East
Willowdale South
Don Mills North
Downsview, North Park, Upwood Park
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
Lawrence Heights, Lawrence Manor
Downsview Northwest
Glencairn
Hillcrest Village
Parkwoods
York Mills West
Newtonbrook, Willowdale
Downsview West
CFB Toronto, Downsview East


#### Check the size of the resulting dataframe

In [22]:
print(northyork_venues.shape)
northyork_venues.head()

(237, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,CF Fairview Mall,43.777548,-79.344485,Shopping Mall
1,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store
2,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Shoppers Drug Mart,43.778878,-79.344697,Pharmacy
3,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Apple Fairview,43.777915,-79.343561,Electronics Store
4,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Hero Certified Burgers,43.777295,-79.344584,Burger Joint


Check how many venues were returned for each neighborhood

In [23]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,4,4,4,4,4,4
Downsview Central,4,4,4,4,4,4
Downsview Northwest,5,5,5,5,5,5
Downsview West,4,4,4,4,4,4
"Downsview, North Park, Upwood Park",4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 108 uniques categories.


# Analysis Each Neighborhood

In [25]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Liquor Store,Lounge,Luggage Store,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Examine new dataframe size

In [26]:
northyork_onehot.shape

(237, 109)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Liquor Store,Lounge,Luggage Store,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.055556,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.00,0.055556,0.055556,0.000000,0.000000,0.055556,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.041667,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.041667,0.041667,0.000000,0.00,0.000000,0.000000,0.083333,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.041667,0.041667,0.000000,0.00,0.041667,0.0,0.00,0.000000,0.000000,0.041667,0.000000,0.0,0.083333,0.000000,0.041667,0.000000,0.041667,0.

### Let's confirm the new size

In [28]:
northyork_grouped.shape

(24, 109)

#### Let's print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                 venue  freq
0          Coffee Shop  0.11
1   Frozen Yogurt Shop  0.06
2        Shopping Mall  0.06
3  Fried Chicken Joint  0.06
4                Diner  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.08
1       Thai Restaurant  0.08
2    Italian Restaurant  0.08
3           Coffee Shop  0.08
4                  Café  0.04


----CFB Toronto, Downsview East----
      venue  freq
0   Airport  0.33
1      Park  0.33
2  Bus Stop  0.33
3     Plaza  0.00
4  Pharmacy  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center  0.25
1  Caribbean Restaurant  0.25
2                  Café  0.25
3   Japanese Restaurant  0.25
4     Martial Arts Dojo 

#### Let's put it into a `pandas` dataframe

Write a function to sort the venues in descending order.

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Bank,Bridal Shop,Restaurant,Sandwich Place,Deli / Bodega,Shopping Mall,Pizza Place,Pharmacy,Fast Food Restaurant
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Thai Restaurant,Italian Restaurant,Fast Food Restaurant,Café,Liquor Store,Butcher,Pharmacy,Pizza Place,Juice Bar
3,"CFB Toronto, Downsview East",Airport,Park,Bus Stop,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Women's Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


# Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [32]:
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 4, 0, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = northyork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northyork_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Food Court,Asian Restaurant,Restaurant,Bakery,Japanese Restaurant,Movie Theater,Metro Station
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Intersection,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,1,Park,Martial Arts Dojo,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
3,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,0,Gym,Asian Restaurant,Coffee Shop,Beer Store,Shopping Mall,Grocery Store,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Discount Store
4,M2R,North York,Willowdale West,43.782736,-79.442259,0,Pizza Place,Butcher,Grocery Store,Coffee Shop,Pharmacy,Furniture / Home Store,Frozen Yogurt Shop,Golf Course,Gift Shop,Comfort Food Restaurant


Finally, let's visualize the resulting clusters

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

### Cluster 1

In [35]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Food Court,Asian Restaurant,Restaurant,Bakery,Japanese Restaurant,Movie Theater,Metro Station
1,North York,0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Intersection,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,North York,0,Gym,Asian Restaurant,Coffee Shop,Beer Store,Shopping Mall,Grocery Store,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Discount Store
4,North York,0,Pizza Place,Butcher,Grocery Store,Coffee Shop,Pharmacy,Furniture / Home Store,Frozen Yogurt Shop,Golf Course,Gift Shop,Comfort Food Restaurant
6,North York,0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
7,North York,0,Home Service,Business Service,Food Truck,Baseball Field,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
8,North York,0,Pizza Place,Furniture / Home Store,Empanada Restaurant,Gift Shop,General Entertainment,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
9,North York,0,Coffee Shop,Thai Restaurant,Italian Restaurant,Fast Food Restaurant,Café,Liquor Store,Butcher,Pharmacy,Pizza Place,Juice Bar
10,North York,0,Coffee Shop,Ramen Restaurant,Pizza Place,Café,Restaurant,Sandwich Place,Movie Theater,Shopping Mall,Plaza,Indonesian Restaurant
11,North York,0,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Women's Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


#### Cluster 2

In [36]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,1,Park,Martial Arts Dojo,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
20,North York,1,Park,Bank,Women's Store,Empanada Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
21,North York,1,Park,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant


#### Cluster 3

In [37]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,North York,2,Furniture / Home Store,Baseball Field,Women's Store,Empanada Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant


#### Cluster 4

In [38]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,3,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
